<a href="https://colab.research.google.com/github/ViswaGanesh-B/GIT-Y/blob/main/final_acc_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
import numpy as np
import random

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Data transforms
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load CIFAR-10 dataset
train_set = CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform_test)

# Labeled and unlabeled split
num_labeled = 4000
indices = list(range(len(train_set)))
random.shuffle(indices)
labeled_subset = Subset(train_set, indices[:num_labeled])

train_loader = DataLoader(labeled_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

# Model definition
class SmallCNN(nn.Module):
    def __init__(self):
        super(SmallCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64 * 8 * 8)
        x = self.dropout(F.relu(self.fc1(x)))
        return self.fc2(x)

# Initialize model, loss function, optimizer
model = SmallCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(350):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_loader):.4f}")

# Evaluation
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

print(f"\n✅ Final Accuracy on CIFAR-10: {100 * correct / total:.2f}%")


Epoch 1: Loss = 2.1366
Epoch 2: Loss = 1.8487
Epoch 3: Loss = 1.7460
Epoch 4: Loss = 1.6855
Epoch 5: Loss = 1.6280
Epoch 6: Loss = 1.5983
Epoch 7: Loss = 1.5628
Epoch 8: Loss = 1.5537
Epoch 9: Loss = 1.5173
Epoch 10: Loss = 1.4897
Epoch 11: Loss = 1.4529
Epoch 12: Loss = 1.4366
Epoch 13: Loss = 1.4186
Epoch 14: Loss = 1.3914
Epoch 15: Loss = 1.3803
Epoch 16: Loss = 1.3584
Epoch 17: Loss = 1.3208
Epoch 18: Loss = 1.3301
Epoch 19: Loss = 1.3069
Epoch 20: Loss = 1.3182
Epoch 21: Loss = 1.3173
Epoch 22: Loss = 1.3189
Epoch 23: Loss = 1.2729
Epoch 24: Loss = 1.2745
Epoch 25: Loss = 1.2653
Epoch 26: Loss = 1.2710
Epoch 27: Loss = 1.2373
Epoch 28: Loss = 1.2565
Epoch 29: Loss = 1.2445
Epoch 30: Loss = 1.2072
Epoch 31: Loss = 1.1961
Epoch 32: Loss = 1.2233
Epoch 33: Loss = 1.1972
Epoch 34: Loss = 1.1944
Epoch 35: Loss = 1.1984
Epoch 36: Loss = 1.1785
Epoch 37: Loss = 1.1705
Epoch 38: Loss = 1.1582
Epoch 39: Loss = 1.1556
Epoch 40: Loss = 1.1760
Epoch 41: Loss = 1.1642
Epoch 42: Loss = 1.1505
E